## **M15 - Cúmulo globular**

Base de código para el cálculo, en las imágenes del HST, de: <br>
- centroide (MÁSCARA PARA NÚCLEO)
- transformación XY - V2V3 - NE (TRANSFORMACIÓN DE COORDENADAS)
- conteo de estrellas aproximadas (CONTEO TOTAL DE ESTRELLAS)
- detección de estrellas más lejanas respecto al núcleo y sus distancias ( DETECCIÓN DE LAS MÁS LEJANAS AL CENTROIDE)
- detección de estrellas más brillantes (DETECCIÓN DE ESTRELLAS BRILLANTES)
- visualización con todo lo anterior, orientación, retícula, ejes... (VISUALIZACIÓN)


In [ ]:
# Librerías necesarias
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
import math
from astropy import units as u
from scipy.ndimage import label, center_of_mass
from skimage.measure import regionprops
from skimage.feature import peak_local_max
from matplotlib.patches import Circle

# Cargar la fuente
font_path = "/usr/share/fonts/msttcore/times.ttf"
tnr_font = fm.FontProperties(fname=font_path)

# Cargar los datos de la imagen
F814W = fits.open('NGC7078/ibv402030_drz.fits')  # FITS file
F814W_h = F814W[0].header
F814W_d = fits.getdata('NGC7078/ibv402030_drz.fits')
F814W_d = np.nan_to_num(F814W_d)  # Reemplazar NaN por 0

# Obtener dimensiones
height, width = F814W_d.shape
print(f"Píxeles de la imagen: {F814W_d.shape}")

# Obtener WCS
wcs = WCS(F814W[1].header, naxis=2)

###################### PASO DE AR - h m s Y DEC º ' " ######################
# Obtener AR y DEC de la cabecera
AR = F814W_h['RA_TARG']
DEC = F814W_h['DEC_TARG']
# Función para convertir grados a horas, minutos, segundos (1h son 15º)
def grados_a_hms(ar_deg):
    horas = int(ar_deg / 15)  # Convertir grados a horas
    minutos = int((ar_deg / 15 - horas) * 60)  # Convertir la parte fraccionaria de horas a minutos
    segundos = ((ar_deg / 15 - horas) * 60 - minutos) * 60  # Convertir la parte fraccionaria de minutos a segundos
    return f"{horas}h {minutos}m {segundos:.2f}s"
# Convertir AR a formato de horas, minutos y segundos
AR_hms = grados_a_hms(AR)

# Función para convertir grados a grados, minutos y segundos
def grados_a_gms(deg):
    grados = int(deg)  # Parte entera de los grados
    minutos = int(abs(deg - grados) * 60)  # Convertir la parte decimal de grados a minutos
    segundos = (abs(deg - grados) * 60 - minutos) * 60  # Convertir la parte decimal de minutos a segundos
    return f"{grados}° {minutos}' {segundos:.2f}'' "

# Convertir DEC a formato de grados, minutos y segundos
DEC_gms = grados_a_gms(DEC)

##################################################################

#####################################   MÁSCARA PARA NÚCLEO   #####################################

#  Definir el radio de interés en píxeles para el núcleo
R_nucleo = 450  # Reducimos el radio para centrarnos más en la zona brillante

#  Obtener el centro de la imagen
center_x, center_y = height // 2, width // 2

#  Crear una máscara circular para limitar la búsqueda del núcleo
Y, X = np.ogrid[:height, :width]
mask_circle_nucleo = (X - center_x) ** 2 + (Y - center_y) ** 2 <= R_nucleo ** 2  # Ecuación del círculo

#  Aplicar la máscara circular a la imagen original
masked_F814W = np.where(mask_circle_nucleo, F814W_d, 0)

#  Aplicar un umbral de brillo SOLO dentro del círculo del núcleo
umbral_nucleo = np.percentile(masked_F814W[mask_circle_nucleo], 75)  # Subimos el umbral para mejor precisión
mask_nucleo = masked_F814W > umbral_nucleo

#  Encontrar las regiones conectadas del núcleo
labeled, num_features = label(mask_nucleo)
region_sizes = np.bincount(labeled.ravel())[1:]  # Contar tamaños de regiones
largest_region_idx = np.argmax(region_sizes) + 1  # Índice de la región más grande
mask_nucleo = labeled == largest_region_idx  # Filtrar solo la región central

#  Obtener el centroide ponderado (núcleo de la nebulosa)
centroid = center_of_mass(masked_F814W * mask_nucleo)
centroid = (int(centroid[0]), int(centroid[1]))  # Convertir a enteros
print('---------------------------------------------------------------------------')
print(f"Centro refinado de M15: {centroid}")

# Convertir a coordenadas celestes
RA_centroid, DEC_centroid = wcs.all_pix2world([[centroid[1], centroid[0]]], 1)[0]
RA_centroid_hms = grados_a_hms(RA_centroid)
DEC_centroid_gms = grados_a_gms(DEC_centroid)
print(f"Centro refinado de M15 en coordendas (α,δ): ({RA_centroid_hms}, {DEC_centroid_gms}) ")
print('----------------------------------------------------------------------------')

#####################################   TRANSFORMACIÓN DE COORDENADAS   #####################################
# Datos de entrada
Xr = 537.0
Yr = 512.0
V2r = -1.2858  # en arcosegundos
V3r = -93.3875 # en arcosegundos
Sx = 0.039568
Sy = 0.039802
beta_x = -41.7091  # en grados
beta_y = 44.5071   # en grados
X = 667.0
Y = 421.0
PA_V3 = 291.587402  # Ángulo PA_V3 en grados

# Conversión de grados a radianes, para poder trabajar con ellos
beta_x_rad = math.radians(beta_x)
beta_y_rad = math.radians(beta_y)

# Convertir PA_V3 a radianes
PA_V3_rad = math.radians(PA_V3)

# Cálculo de V2 arcosegundos
V2 = V2r + Sx * math.sin(beta_x_rad) * (X - Xr) + Sy * math.sin(beta_y_rad) * (Y - Yr)

# Cálculo de V3 arcosegundos
V3 = V3r + Sx * math.cos(beta_x_rad) * (X - Xr) + Sy * math.cos(beta_y_rad) * (Y - Yr)

# Rotación de las coordenadas V2 y V3 al sistema Norte y Este
N = V3 * math.cos(PA_V3_rad) - V2 * math.sin(PA_V3_rad)  # Coordenada Norte relativa
E = V3 * math.sin(PA_V3_rad) + V2 * math.cos(PA_V3_rad)  # Coordenada Este relativa

print('----------------------------------------------------------------------------')
print('')
#####################################   CONTEO TOTAL DE ESTRELLAS   #####################################
# Umbral más bajo para detectar TODAS las estrellas (incluso débiles)
img_norm_all = (F814W_d - np.min(F814W_d)) / (np.max(F814W_d) - np.min(F814W_d))
coords_all = peak_local_max(img_norm_all, min_distance=3, threshold_abs=0.15)  # Ajusta según imagen

print(f"Conteo total aproximado de estrellas en la imagen: {len(coords_all)}")

#####################################   DETECCIÓN DE LAS MÁS LEJANAS AL CENTROIDE   #####################################
# Calcular distancia de cada estrella al centro
distancias = np.linalg.norm(coords_all - np.array(centroid), axis=1)

# Obtener índices de las 5 estrellas más lejanas (puedes ajustar el número)
n = 3
indices_lejanas = np.argsort(distancias)[-n:]
coords_lejanas = coords_all[indices_lejanas]

# Calcular distancia en píxeles y convertir a parsec
pix_distances = distancias[indices_lejanas]

# Escala de píxel en arcsec/pix
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Distancia a M15 en años luz
d_ly = 33600 # sacado de internet
ly_to_pc = 0.3066
d_pc = d_ly * ly_to_pc  # ≈ 103416 pc

# Escala de píxel (arcsec/pix)
pixel_scales = np.abs(wcs.pixel_scale_matrix.diagonal()) * 3600
arcsec_per_pix = np.mean(pixel_scales)

# Distancia en arcsec
dist_arcsec = pix_distances * arcsec_per_pix
tetha_rad = (dist_arcsec / 3600) * (np.pi / 180)
dist_parsec = tetha_rad * d_pc

# Imprimir resultados
for i, (p, a, pc) in enumerate(zip(pix_distances, dist_arcsec, dist_parsec), 1):
    print(f"Estrella {i}: {p:.1f} px ≈ {a:.2f}\" ≈ {pc:.2f} pc")


#####################################   DETECCIÓN DE ESTRELLAS BRILLANTES   #####################################
# Paso 1: Normalización (si no la tienes ya hecha)
img_norm = (F814W_d - np.min(F814W_d)) / (np.max(F814W_d) - np.min(F814W_d))

# Paso 2: Detección de picos locales (estrellas brillantes)
coords_estrellas = peak_local_max(img_norm, min_distance=5, threshold_abs=0.9)  # Puedes ajustar el umbral

# Mensaje informativo
print(f"Se han detectado y marcado las {len(coords_estrellas)} estrellas más brillantes.")

#####################################   VISUALIZACIÓN   #####################################
vmin = np.percentile(F814W_d[F814W_d > 0], 30)   # 5% más oscuro
vmax = np.percentile(F814W_d[F814W_d > 0], 99.9) # 99.5% más brillante
# Crear la figura sin distorsiones
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs)  ## Trabajar en 2D

#  Ajuste del colormap y el contraste
im = ax.imshow(F814W_d, cmap='inferno', norm=LogNorm(vmin=vmin, vmax=vmax))  # Ajustado para más contraste
ax.set_title('M15 - Cúmulo globular (F814W)', fontproperties=tnr_font, fontsize=18, pad=15)
ax.set_xlabel('α', fontproperties=tnr_font, fontsize=12)
ax.set_ylabel('δ', fontproperties=tnr_font, fontsize=12)
#ax.axis('off') 
# Modificar la fuente de los valores de la retícula (RA y DEC)
ax.coords[0].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # α (RA)
ax.coords[1].set_ticklabel(size=12, fontproperties=tnr_font, simplify=False)  # δ (DEC)

# Marcar el centro refinado en la imagen
ax.plot(centroid[1], centroid[0], '+', color='black', markersize=7, label="Centro Refinado")

# Dibujar círculos y líneas para cada estrella
# Dibujar círculos, líneas y etiquetas numeradas para cada estrella lejana
for i, (y, x) in enumerate(coords_lejanas):
    ax.add_patch(Circle((x, y), radius=15, edgecolor='cyan', facecolor='none', linewidth=2))
    ax.plot([centroid[1], x], [centroid[0], y], color='lime', linestyle='--', linewidth=1.2)
    ax.text(x + 12, y, f'{i+1}', color='white', fontsize=13, fontproperties=tnr_font,
            ha='left', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=1.5))


#  Marcar estrellas más brillantes
for y, x in coords_estrellas:
    circ = Circle((x, y), radius=12, edgecolor='green', facecolor='none', linewidth=1.5)
    ax.add_patch(circ)

# Añadir anotación para AR y DEC
text_str = f"Coordenadas del núcleo\nα: {grados_a_hms(RA_centroid)}\nδ: {grados_a_gms(DEC_centroid)}"
ax.text(0.95, 0.95, text_str, transform=ax.transAxes, color='white', fontsize=12,
        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# Añadir anotación para N y E
#text_str = f"Nc: {N:.6f} (arcsec)\nEc: {E:.6f} (arcsec)"
#ax.text(0.95, 0.855, text_str, transform=ax.transAxes, color='white', fontsize=10,
#        fontweight='bold', fontproperties=tnr_font, ha='right', va='top', bbox=dict(facecolor='black', alpha=0.5))

# --------------------------
# Añadir Norte y Este en la retícula correcta
# Punto en la esquina inferior derecha para la orientación
x, y = 975, 140.5
ra_dec_corner = wcs.pixel_to_world(x, y)

# Desplazamientos para las flechas
delta_ra = (0.0011 * u.deg)
delta_dec = (0.0011 * u.deg)

# Calcular coordenadas de Norte y Este
ra_dec_norte = ra_dec_corner.spherical_offsets_by(0 * u.deg, delta_dec)
ra_dec_este = ra_dec_corner.spherical_offsets_by(delta_ra, 0 * u.deg)

# Convertir de vuelta a píxeles
norte_x, norte_y = wcs.world_to_pixel(ra_dec_norte)
este_x, este_y = wcs.world_to_pixel(ra_dec_este)

# Dibujar flechas en la dirección correcta
ax.arrow(x, y, norte_x - x, norte_y - y, 
         color='red', head_width=20, head_length=15, label="Norte")
ax.arrow(x, y, este_x - x, este_y - y, 
         color='red', head_width=20, head_length=15, label="Este")

# Etiquetas
ax.text(norte_x + 25, norte_y, 'N', color='red', fontsize=14, fontproperties=tnr_font, fontweight='bold', ha='center', va='bottom')
ax.text(este_x, este_y - 40, 'E', color='red', fontsize=14, fontproperties=tnr_font, fontweight='bold', ha='center', va='bottom')


# Añadir la barra de color
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Brillo', fontproperties=tnr_font, fontsize=12)
cbar.ax.tick_params(labelsize=10)
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(tnr_font)


# Configurar la retícula correctamente
ax.grid(color='white', linestyle='--', linewidth=0.7)
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)  # Ajuste del espacio

plt.show()
F814W.close()
